In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')
from bs4 import BeautifulSoup
from lxml import etree
import requests

In [6]:
headers = {
    'x-devtools-emulate-network-conditions-client-id': "c60c3ba6-26e8-45ce-9f9f-0b5521437cf6",
    'upgrade-insecure-requests': "1",
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36",
    'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    'referer': "http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247",
    'accept-encoding': "gzip, deflate, sdch",
    'accept-language': "es-ES,es;q=0.8",
    'cookie': "LANGUE_LOGIN=fr",
    'cache-control': "no-cache"
}

data = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247", headers=headers)

soup = BeautifulSoup(data.content, 'html.parser')
queries = soup.find_all('select')
queries_map = {}
for s in queries:
    queries_map[s.get('name')] = {}
    map = queries_map[s.get('name')]
    options = s.find_all('option')
    for o in options:
        if not o.string:
            value = ""
        else:
            value = o.string
        map[value] = o.get('value')

print(queries_map)

{'ww_x_PERIODE_ACAD': {'': 'null', '2014-2015': '213637922', '2009-2010': '978195', '2016-2017': '355925344', '2015-2016': '213638028', '2011-2012': '123455150', '2012-2013': '123456101', '2008-2009': '978187', '2013-2014': '213637754', '2010-2011': '39486325', '2007-2008': '978181'}, 'ww_x_UNITE_ACAD': {'': 'null', 'Génie civil': '942623', 'Chimie et génie chimique': '246696', 'Génie électrique et électronique ': '943936', 'Architecture': '942293', 'Informatique': '249847', 'Management de la technologie': '946882', 'Génie mécanique': '944263', 'EME (EPFL Middle East)': '637841336', 'Section FCUE': '1574548993', 'Microtechnique': '945244', 'Cours de mathématiques spéciales': '943282', "Sciences et ingénierie de l'environnement": '942953', 'Systèmes de communication': '946228', 'Science et génie des matériaux': '944917', 'Sciences et technologies du vivant': '945901', 'Mathématiques': '944590', 'Humanités digitales': '2054839157', 'Physique': '945571', 'Ingénierie financière': '12062311

In [17]:
def getData(unite_acad, periode_acad, periode_pedago, semester_type):
    payload = {'ww_x_GPS': -1,
               'ww_i_reportModel':133685247, 
               'ww_i_reportModelXsl': 133685271,
               'ww_x_UNITE_ACAD': unite_acad,
               'ww_x_PERIODE_ACAD': periode_acad,
               'ww_x_PERIODE_PEDAGO': periode_pedago,
               'ww_x_HIVERETE': semester_type
              }
    r = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.XLS", params=payload)
    return r.text

In [24]:
total_data = pd.DataFrame()

def addData(unite_acad, periode_acad, periode_pedago, semester_type):
    unite_acad = queries_map['ww_x_UNITE_ACAD'][unite_acad]
    periode_acad = queries_map['ww_x_PERIODE_ACAD'][periode_acad]
    periode_pedago = queries_map['ww_x_PERIODE_PEDAGO'][periode_pedago]
    semester_type = queries_map['ww_x_HIVERETE'][semester_type]

    
    soup = BeautifulSoup(getData(unite_acad,periode_acad,periode_pedago,semester_type), 'html.parser')
    rows = soup.find_all('tr')
    header_cols = rows[2].find_all('th')
    col_titles = []
    data = {}
    for c in header_cols:
        data[c.string] = []
        col_titles.append(c.string)
    df = pd.DataFrame(columns=col_titles)
    for h in range(3, len(rows)):
        values = rows[h].find_all('td')
        for v in range(len(values)-2):
            data[header_cols[v].string].append(values[v].string)
    df = pd.DataFrame.from_dict(data)
    return df



In [25]:

unite_acad = "Informatique"
periode_acad = list(queries_map['ww_x_PERIODE_ACAD'].keys())
periode_acad.remove('')
periode_pedago = []
for p in list(queries_map['ww_x_PERIODE_PEDAGO'].keys()):
    if 'Bachelor' in p:
        periode_pedago.append(p)
a = addData("Informatique", "2008-2009", "Bachelor semestre 2", "")
a


,Civilité,Ecole Echange,Filičre opt.,Mineur,No Sciper,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Statut,Type Echange
0,Monsieur,None,None,None,184228,Ahluwalia Samit,None,None,None,Présent,None
1,Monsieur,None,None,None,179383,Amiet Nils,None,None,None,Présent,None
2,Monsieur,None,None,None,184772,Arnfred Jonas,None,None,None,Présent,None
3,Monsieur,None,None,None,186264,Bougacha Malik,None,None,None,Présent,None
4,Monsieur,None,None,None,185949,Bourgeois Jérémy,None,None,None,Présent,None
5,Monsieur,None,None,None,184264,Bourqui Marc,None,None,None,Présent,None
6,Monsieur,None,None,None,186825,Bruchez Lionel,None,None,None,Présent,None
7,Monsieur,None,None,None,179864,Cambazard Thomas,None,None,None,Présent,None
8,Monsieur,None,None,None,184991,Chablais Michael,None,None,None,Présent,None
9,Monsieur,None,None,None,185283,Constantin Victor,None,None,None,Présent,None
